In [1]:
#set the API key
from tmdbv3api import TMDb
from tmdbv3api import Movie
tmdb = TMDb()
movie = Movie()
tmdb.api_key = 'c6cd98753f10dc390a8c4a5b4b83d47c'
tmdb.language = 'en'
tmdb.debug = True

In [2]:
import requests


In [3]:
needed_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
response = requests.get("https://www.themoviedb.org/tv", headers = needed_headers)


In [4]:
response.status_code

200

In [5]:
dwn_content = response.text
len(dwn_content)

193830

In [10]:
dwn_content[:500]
'<!DOCTYPE html>'
\n  <html lang="en" class="no-js">
\n  <head>
\n  <title>Popular TV Shows — The Movie Database (TMDB)</title>
\n  <meta http-equiv="X-UA-Compatible" content="IE=edge" />
\n  <meta http-equiv="cleartype" content="on">
\n  <meta charset="utf-8">
\n  
\n  <meta name="keywords" content="Movies, TV Shows, Streaming, Reviews, API, Actors, Actresses, Photos, User Ratings, Synopsis, Trailers, Teasers, Credits, Cast">
\n  <meta name="mobile-web-app-capable" content="yes">
\n  <meta name="'

SyntaxError: unexpected character after line continuation character (3556955472.py, line 3)

In [12]:
dwn_content[:500]
'<!DOCTYPE html>'

'<!DOCTYPE html>'

In [27]:
\n <html lang="en">


SyntaxError: unexpected character after line continuation character (4023631606.py, line 1)